In [ ]:
!pip install opencv-python-headless
!pip install mediapipe


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import files


uploaded = files.upload()
vid_path = list(uploaded.keys())[0]

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.5)


cap = cv2.VideoCapture(vid_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to save the video
output_path = 'output_bottle_flip_detection.avi'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width, frame_height))

if not cap.isOpened():
    print("Error opening video file")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to find hands
        results = hands.process(rgb_frame)

        hand_count = 0
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                hand_count += 1

        # Check if more than one hand is detected
        if hand_count > 1:
            label = "Invalid Flip (Cheat Detected)"
            color = (0, 0, 255)  # Red for invalid
        else:
            label = "Valid Flip"
            color = (0, 255, 0)  # Green for valid

        # Display the result on the frame
        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)


        out.write(frame)


        cv2_imshow(frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cap.release()
out.release()
cv2.destroyAllWindows()

# Provide a download link for the output video
files.download(output_path)
